In [4]:
import random

import boto3
import pandas as pd
import polars as pl
import numpy as np
import scipy.sparse as sp

from tqdm import tqdm
from typing import List, Any
from collections import defaultdict

import faiss
import optuna
import implicit
from gensim.models import Word2Vec

import constants
from utils.io.s3 import DataFrameType, download_dataframe

/opt/conda/envs/recsys/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Читаем датасет

In [5]:
s3_session = boto3.session.Session()
s3_client = s3_session.client(service_name='s3', endpoint_url='https://storage.yandexcloud.net')

In [6]:
data = download_dataframe(s3_client,
                          constants.S3_BUCKET,
                          constants.DATA_PATH / 'dataset.parquet',
                          DataFrameType.POLARS)
# data = pl.read_parquet('train.parquet')
data

user_id,artist_id
str,str
"""d705b538-1bd8-…","""69c71d72-7ed8-…"
"""d705b538-1bd8-…","""30bf469f-9abd-…"
"""d705b538-1bd8-…","""a26c9335-2459-…"
"""d705b538-1bd8-…","""69c903b5-dff0-…"
"""d705b538-1bd8-…","""af8eef9d-13aa-…"
"""d705b538-1bd8-…","""293a86ee-6ce7-…"
"""d705b538-1bd8-…","""348f4909-1c48-…"
"""d705b538-1bd8-…","""ad2bf122-726e-…"
"""d705b538-1bd8-…","""cc97fc57-30b5-…"


## Метрики

В этом задании нашей задачей будем оптимизация метрик ndcg@20. Тем не менее, такая метрику сложно интерпретировать и поэтому вам также будет доступно значение метрики hitrate@20, с которой вы уже познакомились в прошлой домашке

In [7]:
TOP_K = 20


def user_hitrate(y_relevant: List[str], y_recs: List[str], k: int = TOP_K) -> int:
    return int(len(set(y_relevant).intersection(y_recs[:k])) > 0)

def user_ndcg(y_rel: List[Any], y_rec: List[Any], k: int = 10) -> float:
    """
    :param y_rel: relevant items
    :param y_rec: recommended items
    :param k: number of top recommended items
    :return: ndcg metric for user recommendations
    """
    dcg = sum([1. / np.log2(idx + 2) for idx, item in enumerate(y_rec[:k]) if item in y_rel])
    idcg = sum([1. / np.log2(idx + 2) for idx, _ in enumerate(zip(y_rel, np.arange(k)))])
    return dcg / idcg

В этом датасете идентификаторы представлены в виде строк, но для работы с ними может быть проще сделать преобразование в числа (например, для алгоритмов матричной факторизации)

In [8]:
user_mapping = {k: v for v, k in enumerate(data['user_id'].unique())}
user_mapping_inverse = {k: v for v, k in user_mapping.items()}

artist_mapping = {k: v for v, k in enumerate(data['artist_id'].unique())}
artist_mapping_inverse = {k: v for v, k in artist_mapping.items()}

In [9]:
grouped_df_with_inds = (
    data
    .with_columns([
        pl.col('user_id').apply(user_mapping.get),
        pl.col('artist_id').apply(artist_mapping.get),
    ])
    # для каждого пользователя оставим последние 3 объекта в качестве тестовой выборки,
    # а остальное будем использовать для тренировки
    .groupby('user_id')
    .agg([
        pl.col('artist_id').apply(lambda x: x[:-3]).alias('train_item_ids'),
        pl.col('artist_id').apply(lambda x: x[-3:]).alias('test_item_ids'),
    ])
)

grouped_df_with_inds

user_id,train_item_ids,test_item_ids
i64,list[i64],list[i64]
7968,"[35676, 42835, … 38681]","[83125, 40959, 30194]"
40192,"[44706, 66455, … 5799]","[34568, 26, 9761]"
49536,"[84680, 56057, … 68680]","[70157, 65724, 35864]"
27488,"[82918, 41266, … 53024]","[52762, 77785, 60505]"
41088,"[48242, 65461, … 51517]","[62374, 6418, 72919]"
19488,"[84363, 81536, … 5517]","[84023, 52613, 48767]"
18528,"[42783, 6124, … 79587]","[25464, 23277, 76781]"
12832,"[23291, 57858, … 66094]","[72975, 53184, 1155]"
32160,"[84881, 39028, … 70461]","[46136, 15265, 51151]"


In [10]:
# соберем строчки для разреженной матрицы для тренировки
rows = []
cols = []
values = []
for user_id, train_ids, _ in grouped_df_with_inds.rows():
    rows.extend([user_id] * len(train_ids))
    values.extend([1] * len(train_ids))
    cols.extend(train_ids)

train_user_item_data = sp.csr_matrix((values, (rows, cols)))

In [11]:
# соберем строчки для разреженной матрицы со всеми рейтингами
rows = []
cols = []
values = []
for user_id, train_ids, test_ids in grouped_df_with_inds.rows():
    # используем все данные для финальных предсказаний
    train_ids = (train_ids + test_ids)
    rows.extend([user_id] * len(train_ids))
    values.extend([1] * len(train_ids))
    cols.extend(train_ids)

user_item_data = sp.csr_matrix((values, (rows, cols)), dtype=float)

In [12]:
median_seq_len = int(grouped_df_with_inds['train_item_ids'].apply(len).median())
print(f"средняя длина сессии {median_seq_len}")

средняя длина сессии 42


In [13]:
RANDOM_STATE = 42

def set_seed():
    random.seed(RANDOM_STATE)
    np.random.seed(RANDOM_STATE)
    
    
def get_recommendations(user_embs: np.array, item_embs: np.array, k: int = TOP_K):
    # строим индекс объектов
    index = faiss.IndexFlatIP(item_embs.shape[1])
    index.add(item_embs)

    # строим рекомендации с помощью dot-product расстояния
    # с запасом, чтобы после фильтрации просмотренных осталось хотя бы TOP_K
    return index.search(user_embs, k)

## Бейзлайны

В качестве простого бейзлайна будем рекомендовать самый популярных артистов

Мы хотим сначала провалидировать такое решение, а значит в качестве популярных артистов мы возьмем только тех, кто чаще встречается в `train_item_ids`

In [13]:
top_artists = (
    grouped_df_with_inds
    .select(pl.col('train_item_ids').alias('artist_id'))
    .explode('artist_id')
    .groupby('artist_id')
    .count()
    .sort('count', descending=True)
    .head(1_000)
)['artist_id'].to_list()

In [14]:
ndcg_list = []
hitrate_list = []

for user_id, user_history, y_rel in grouped_df_with_inds.rows():
    y_rec = top_artists.copy()[:TOP_K]
    
    ndcg_list.append(user_ndcg(y_rel, y_rec))
    hitrate_list.append(user_hitrate(y_rel, y_rec))
    
print(f'NDCG@{TOP_K} = {np.mean(ndcg_list):.5f}, Hitrate@{TOP_K} = {np.mean(hitrate_list):.5f}')

NDCG@20 = 0.01401, Hitrate@20 = 0.10248


Не забываем про фильтрацию просмотренного (для разных доменов и подходов это не всегда улучши рекомендации, но в данном случае дало прирост)

In [15]:
ndcg_list = []
hitrate_list = []

for user_id, user_history, y_rel in grouped_df_with_inds.rows():
    y_rec = [artist_id for artist_id in top_artists if artist_id not in user_history]
    
    ndcg_list.append(user_ndcg(y_rel, y_rec))
    hitrate_list.append(user_hitrate(y_rel, y_rec))
    
print(f'NDCG@{TOP_K} = {np.mean(ndcg_list):.5f}, Hitrate@{TOP_K} = {np.mean(hitrate_list):.5f}')

NDCG@20 = 0.01740, Hitrate@20 = 0.11684


## Построим файл с рекомендациями

Для построения рекомендаций теперь можем учесть все возможные данные

In [16]:
top_artists = (
    data
    .groupby('artist_id')
    .count()
    .sort('count', descending=True)
    .head(1_000)
)['artist_id'].to_list()

In [22]:
submission = []

for user_id, user_history in data.groupby('user_id').agg(pl.col('artist_id')).rows():
    y_rec = [
        artist_id
        for artist_id in top_artists
        if artist_id not in user_history
    ][:TOP_K]
    
    submission.append((user_id, y_rec))
    
submission = pl.DataFrame(submission, schema=('user_id', 'y_rec'))
submission.write_parquet('sample_submission.parquet')
submission

user_id,y_rec
str,list[str]
"""680f12f8-b830-…","[""5cd0ffb5-0cf2-4ecd-8c5b-ca2102e33198"", ""12289298-d9dc-4b1d-bc27-16480829de75"", … ""f468c554-1cf2-4bd6-9281-4ed93216427c""]"
"""271ca035-3a9b-…","[""5cd0ffb5-0cf2-4ecd-8c5b-ca2102e33198"", ""12289298-d9dc-4b1d-bc27-16480829de75"", … ""f468c554-1cf2-4bd6-9281-4ed93216427c""]"
"""2a68fcf1-a5ba-…","[""5cd0ffb5-0cf2-4ecd-8c5b-ca2102e33198"", ""12289298-d9dc-4b1d-bc27-16480829de75"", … ""f468c554-1cf2-4bd6-9281-4ed93216427c""]"
"""9d209d9a-1d8c-…","[""12289298-d9dc-4b1d-bc27-16480829de75"", ""3a6df691-648d-467d-995e-eb06c53ea725"", … ""40d5f4f7-3560-4638-a562-055013eee0d4""]"
"""8718fe5c-48f2-…","[""5cd0ffb5-0cf2-4ecd-8c5b-ca2102e33198"", ""12289298-d9dc-4b1d-bc27-16480829de75"", … ""f468c554-1cf2-4bd6-9281-4ed93216427c""]"
"""9d7b0cf8-23df-…","[""5cd0ffb5-0cf2-4ecd-8c5b-ca2102e33198"", ""3a6df691-648d-467d-995e-eb06c53ea725"", … ""f0c7d12c-a479-4d43-b786-51e6e28bad59""]"
"""cc92d25d-5cf0-…","[""5cd0ffb5-0cf2-4ecd-8c5b-ca2102e33198"", ""3a6df691-648d-467d-995e-eb06c53ea725"", … ""42cee962-0f50-4728-b887-01cb7a207075""]"
"""42196377-02d8-…","[""5cd0ffb5-0cf2-4ecd-8c5b-ca2102e33198"", ""12289298-d9dc-4b1d-bc27-16480829de75"", … ""40d5f4f7-3560-4638-a562-055013eee0d4""]"
"""e7474f77-1995-…","[""5cd0ffb5-0cf2-4ecd-8c5b-ca2102e33198"", ""ee0f3f04-8fa0-46ad-b821-2a5ebf5ca6e9"", … ""87a7508f-415e-4080-8bff-8ff94cfec4a6""]"


Не забывайте, что в файле с рекомендациями должны быть **исходные идентификаторы (строки)**, а не преобразованные в числа!

## ALS

Модель показала лучшие результаты с подобранными с помощью `optuna` гиперпараметрами

In [46]:
top_artists_mapped = [artist_mapping[artist_id] for artist_id in top_artists]

def evaluate_model(model):
    # строим рекомендации в виде KNN алгоритма поверх эмбеддингов пользователей и артистов
    _, recs = get_recommendations(
        model.user_factors,
        model.item_factors,
        TOP_K + median_seq_len
    )
    
    # будем отслеживать как ndcg, так и hitrate метрики
    ndcg_list = []
    hitrate_list = []
    
    for user_id, user_history, y_rel in grouped_df_with_inds.select(
        'user_id', 'train_item_ids', 'test_item_ids'
    ).rows():
        user_history = set(user_history)
        if user_id < len(recs):
            y_rec = [
                item_id
                # чтобы точно хватило рекомендаций, добавим еще топовых артистов в конец списка
                for item_id in list(recs[user_id]) + top_artists_mapped
                if item_id not in user_history
            ]
        else:
            y_rec = []
        hitrate_list.append(user_hitrate(y_rel, y_rec))
        ndcg_list.append(user_ndcg(y_rel, y_rec))
        
    mean_ndcg = np.mean(ndcg_list)
    mean_hitrate = np.mean(hitrate_list)
    return mean_ndcg, mean_hitrate

In [33]:
def objective(trial):
    factors = trial.suggest_int('factors', 8, 64)
    iterations = trial.suggest_int('iterations', 5, 30)
    alpha = trial.suggest_float('alpha', 0.1, 5.0)
    regularization = trial.suggest_float('regularization', 1e-3, 1e-1)
        
    print({
        'factors': factors,
        'iterations': iterations,
        'alpha': alpha,
        'regularization': regularization,
    })
    
    set_seed()
    als_model = implicit.als.AlternatingLeastSquares(
        factors=factors,
        iterations=iterations,
        random_state=RANDOM_STATE,
        alpha=alpha,
        regularization=regularization
    )
    als_model.fit(train_user_item_data)
    
    mean_ndcg, mean_hitrate = evaluate_model(als_model)
    print(f'NDCG@{TOP_K} = {mean_ndcg}, Hitrate@{TOP_K} = {mean_hitrate}')
    return mean_ndcg
    
    
study = optuna.create_study(directions=('maximize',))
# тут запускается всего 5 итераций, что может быть очень мало для хороших результатов
study.optimize(objective, n_trials=5)

study.best_params

[I 2023-10-16 17:18:32,470] A new study created in memory with name: no-name-e2981e4b-f26f-4135-a08b-08a010f20ee3


{'factors': 36, 'iterations': 9, 'alpha': 2.3789267986759275, 'regularization': 0.07534946665601996}


  0%|          | 0/9 [00:00<?, ?it/s]

[I 2023-10-16 17:18:58,217] Trial 0 finished with value: 0.05601240968411231 and parameters: {'factors': 36, 'iterations': 9, 'alpha': 2.3789267986759275, 'regularization': 0.07534946665601996}. Best is trial 0 with value: 0.05601240968411231.


NDCG@20 = 0.05601240968411231, Hitrate@20 = 0.32418
{'factors': 29, 'iterations': 20, 'alpha': 3.387909934029769, 'regularization': 0.018921582625997854}


...

NDCG@20 = 0.04873747434148573, Hitrate@20 = 0.28692
{'factors': 12, 'iterations': 13, 'alpha': 1.260570402495505, 'regularization': 0.09605942395989482}


  0%|          | 0/13 [00:00<?, ?it/s]

[I 2023-10-16 17:20:43,015] Trial 4 finished with value: 0.043673272588193195 and parameters: {'factors': 12, 'iterations': 13, 'alpha': 1.260570402495505, 'regularization': 0.09605942395989482}. Best is trial 0 with value: 0.05601240968411231.


NDCG@20 = 0.043673272588193195, Hitrate@20 = 0.26282


{'factors': 36,
 'iterations': 9,
 'alpha': 2.3789267986759275,
 'regularization': 0.07534946665601996}

In [34]:
# одни из оптимальных параметров (у вас может получиться еще лучше)
params = {
    'factors': 128,
    'iterations': 40,
    'alpha': 5,
    'regularization': 0.02,
    'random_state': RANDOM_STATE
}

als_model = implicit.als.AlternatingLeastSquares(
    **params
)
als_model.fit(train_user_item_data)

mean_ndcg, mean_hitrate = evaluate_model(als_model)
print(f'NDCG@{TOP_K} = {mean_ndcg}, Hitrate@{TOP_K} = {mean_hitrate}')

  0%|          | 0/40 [00:00<?, ?it/s]

NDCG@20 = 0.06642426190347041, Hitrate@20 = 0.36858


Для финальных рекомендаций хорошо бы использовать все данные, поэтому обучим алгоритм на полных данных с теми же гиперпараметрами

In [14]:
sample_submission = (
    pl.read_parquet('sample_submission.parquet')
    .with_columns([pl.col('user_id').apply(user_mapping.get)])
)

# используем все имеющиеся данные для обучения модели
train_data = (
    data
    .with_columns([
        pl.col('user_id').apply(user_mapping.get),
        pl.col('artist_id').apply(artist_mapping.get),
    ])
    .groupby('user_id').agg(pl.col('artist_id').alias('user_history'))
    .join(sample_submission, 'user_id', 'inner')
    .select('user_id', 'user_history')
)

In [47]:
als_model = implicit.als.AlternatingLeastSquares(
    **params
)
als_model.fit(user_item_data)

_, recs = get_recommendations(
    als_model.user_factors,
    als_model.item_factors,
    TOP_K + median_seq_len
)

submission = []
for user_id, user_history in tqdm(train_data.rows()):
    user_history = set(user_history)
    y_rec = [
        artist_mapping_inverse[item_id]
        for item_id in list(recs[user_id]) + top_artists_mapped
        if item_id not in user_history
    ][:TOP_K]
        
    submission.append((user_mapping_inverse[user_id], y_rec))
    
submission = pl.DataFrame(submission, schema=('user_id', 'y_rec'))
submission.write_parquet('als_submission.parquet')
submission

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [00:02<00:00, 18007.58it/s]


user_id,y_rec
str,list[str]
"""1d664c61-55cb-…","[""db2802ef-5aac-4a78-a688-f022944f186b"", ""62c59774-4d7d-4ad5-b46b-e5de9e0730da"", … ""af48c579-07bf-4bab-9755-31d8aaba3511""]"
"""f680183f-5c16-…","[""b306c596-f83f-4007-a4c9-d72ef951e8e8"", ""92f2e07f-24b5-41bd-9d87-15c355cbba08"", … ""2e5f8198-d76c-40de-8c1a-4fda388086c3""]"
"""ad3dbca0-9eb0-…","[""5bc90a5d-1356-4958-821c-64328ac567ea"", ""7a6eba6b-e565-4e9c-a383-00eec0c73f5a"", … ""d5d62e25-f4a8-4b4f-bc98-3fe25346d38d""]"
"""218523a8-2e9d-…","[""24962209-ca4e-4a1e-9e85-fd7d1e86a07f"", ""78ca3640-59ca-4e18-b218-f8605c2c7344"", … ""1cf28489-fb9a-4732-91c1-609b6faeca1d""]"
"""84f5c9be-99a5-…","[""68acfdf7-29cc-4fb0-bfaf-a1a7eecb8502"", ""f4ff906c-7acd-4235-b62d-d8ffc5fbebc7"", … ""1ca54366-2198-4079-a6c5-f030712fe9f3""]"
"""53d0e85b-588a-…","[""e4feec41-ac39-4358-bee5-7608abb8946e"", ""5fa9bef8-ed42-421a-ab82-5e3ad35c0cbb"", … ""e0e8fa50-748b-4e83-baaf-d5daf051e76d""]"
"""b9a43f13-213a-…","[""c8822a9f-f6f2-4025-ae84-8b7cd45c916f"", ""42cee962-0f50-4728-b887-01cb7a207075"", … ""98e69a29-ee83-41f1-924e-08a50a32efdc""]"
"""e62c7b73-54f1-…","[""02e9fac4-b25a-4779-b51b-cefd2189bd59"", ""1657c161-1e2d-4823-b9bc-39db72c4b4d6"", … ""8caad1cc-e3a0-48f5-af61-89b6025d38e0""]"
"""d7f1921d-a86b-…","[""f3dce478-ec1a-45a8-923d-59ecdbc6c59d"", ""58389edf-72df-468f-8e79-7fb8ce6a6124"", … ""2099bb7f-bcb2-40b6-a58c-77f57d375b7d""]"


Такая модель уже выбиывает NDCG@20 = 0.8, но мы пойдем дальше

## W2V

In [21]:
def evaluate_model(model, filter_watched: bool = True):
    ndcg_list = []
    hitrate_list = []

    set_seed()
    for user_history, y_rel in tqdm(grouped_df_with_inds.sample(n=10_000).select(
        'train_item_ids', 'test_item_ids'
    ).rows()):
        model_preds = model.predict_output_word(
            user_history[-model.window:],
            topn=(TOP_K + len(user_history))
        )
        if model_preds is None:
            y_rec = []
        else:
            y_rec = [
                pred[0]
                for pred in model_preds
                if not filter_watched or pred[0] not in user_history
            ]

        ndcg_list.append(user_ndcg(y_rel, y_rec))
        hitrate_list.append(user_hitrate(y_rel, y_rec))
    return np.mean(hitrate_list), np.mean(ndcg_list)

In [ ]:
def objective(trial):
    sg = trial.suggest_categorical('sg', [0, 1])
    window = trial.suggest_int('window', 1, 10)
    ns_exponent = trial.suggest_float('ns_exponent', -3, 3)
    negative = trial.suggest_int('negative', 3, 20)
    min_count = trial.suggest_int('min_count', 0, 20)
    vector_size = trial.suggest_categorical('vector_size', [16, 32, 64, 128])
    
    print({
        'sg': sg,
        'window_len': window,
        'ns_exponent': ns_exponent,
        'negative': negative,
        'min_count': min_count,
        'vector_size': vector_size,
    })
    
    set_seed()
    model = Word2Vec(
        grouped_df_with_inds['train_item_ids'].to_list(),
        window=window,
        sg=sg,
        hs=0,
        min_count=min_count,
        vector_size=vector_size,
        negative=negative,
        ns_exponent=ns_exponent,
        seed=RANDOM_STATE,
        epochs=10,
    )
    
    mean_hitrate, mean_ndcg = evaluate_model(model)
    print(f'NDCG@{TOP_K} = {mean_ndcg}, Hitrate@{TOP_K} = {mean_hitrate}')
    return mean_ndcg
    
    
study = optuna.create_study(directions=('maximize',))
study.optimize(objective, n_trials=5)

study.best_params

In [22]:
model = Word2Vec(
    grouped_df_with_inds['train_item_ids'].to_list(),
    sg=0,
    vector_size=32,
    min_count=18,
    epochs=30,
    negative=17,
    window=10,
    ns_exponent=0.05,
    seed=RANDOM_STATE,
)

mean_hitrate, mean_ndcg = evaluate_model(model)
print(f'NDCG@{TOP_K} = {mean_ndcg}, Hitrate@{TOP_K} = {mean_hitrate}')

100%|██████████| 50000/50000 [00:23<00:00, 2106.77it/s]

NDCG@20 = 0.055886763416826704, Hitrate@20 = 0.32918


In [23]:
model = Word2Vec(
    train_data['user_history'].to_list(),
    sg=0,
    vector_size=32,
    min_count=18,
    epochs=30,
    negative=17,
    window=10,
    ns_exponent=0.05,
    seed=RANDOM_STATE,
)

In [24]:
submission = []
for user_id, user_history in tqdm(train_data.rows()):
    model_preds = model.predict_output_word(
        user_history[-model.window:],
        topn=(TOP_K + len(user_history))
    )
    if model_preds is not None:
        y_rec = [
            artist_mapping_inverse[pred[0]]
            for pred in model_preds
            if pred[0] not in user_history
        ][:TOP_K]
        
    submission.append((user_mapping_inverse[user_id], y_rec))
    
submission = pl.DataFrame(submission, schema=('user_id', 'y_rec'))
submission.write_parquet('w2v_submission.parquet')
submission

100%|██████████| 50000/50000 [00:32<00:00, 1552.32it/s]


user_id,y_rec
str,list[str]
"""680f12f8-b830-…","[""af1969e0-5472-4c3e-8b16-0ff9bcc5092e"", ""3ba7c3fe-009d-476e-a7d2-48d90dbe1670"", … ""7611fde9-6085-4130-89b5-ee3f3e7ea9f9""]"
"""271ca035-3a9b-…","[""929c116f-f9c7-49de-8736-5d5fbed9684b"", ""270e883c-4647-4b35-8768-5d61842337ab"", … ""2c129971-c111-4262-a3ca-d937f0c81e09""]"
"""2a68fcf1-a5ba-…","[""b306c596-f83f-4007-a4c9-d72ef951e8e8"", ""1d763d72-7915-46df-9924-5a8d8b3c5da6"", … ""e0d0391a-7454-4d3e-a690-950204ef59bf""]"
"""9d209d9a-1d8c-…","[""5a0069a4-fd89-4d82-adb8-75f637fb63c5"", ""e2302565-709f-42ba-9df1-8f80a8438c4d"", … ""16f10fa5-0bf4-465a-a190-6475b531139f""]"
"""8718fe5c-48f2-…","[""453b497b-1763-4e49-a7c0-dc9dfbdbbd65"", ""7611fde9-6085-4130-89b5-ee3f3e7ea9f9"", … ""305a1203-f2cc-4ee8-8d71-ed95373a9502""]"
"""9d7b0cf8-23df-…","[""936ecd0f-d6a4-422c-8db9-f3e04066a8e1"", ""ee0f3f04-8fa0-46ad-b821-2a5ebf5ca6e9"", … ""d7cdd1aa-57b6-4bfd-a17a-d30c6f6f0bc3""]"
"""cc92d25d-5cf0-…","[""11494b56-e03a-418a-87ce-bb15b61bd7d4"", ""41fc8889-8d69-4617-88b8-734463af3b70"", … ""5d154542-b2f5-448c-b2ce-9083d3a0ed9f""]"
"""42196377-02d8-…","[""9bb2a8fd-ead3-449d-9a03-8f2c55d606b5"", ""2b5be6d8-bcdc-4ab4-8f5e-d52b742b15d1"", … ""16178a7c-44ae-42be-a8df-0fa162bc2b7c""]"
"""e7474f77-1995-…","[""c0f5fbff-2648-430d-9cc9-c377421e8690"", ""24962209-ca4e-4a1e-9e85-fd7d1e86a07f"", … ""ef363177-3d87-4d6f-9206-0363066adeea""]"


## Объединим методы

В качестве бейзлайнов для объединения нескольких методов может выступать поочередная конкатенация списков, например:

- `y_rec_1 = [1, 2, 3]`
- `y_rec_2 = [4, 5, 6]`

Тогда `y_rec = [1, 4, 2, 5, 3, 6]`

Можно придумать и посложнее метод, например объединять отсортировать все объекты с весами, которые зависят от позиции в списке и качества всего метода

В качестве веса позиции $i$ мы будем использовать $\dfrac{1}{\log(i)}$, что соответствует весу в метрике NDCG

In [78]:
temperature = 40

submissions = ['w2v', 'als']
# веса получены по рузальтатам оценки в LMS
submission_weights = np.array([0.0706, 0.0815])
# в качестве итоговых весов используется softmax с температурой
# чем выше температура, тем больше будет перекос в сторону сильного решения
submission_weights = np.exp(submission_weights * temperature) / \
    np.exp(submission_weights * temperature).sum()
print(submission_weights)

submissions_df = [
    (
        pl.read_parquet(f'{submission}_submission.parquet')
        .with_columns([pl.col('y_rec').alias(f'{submission}_rec')])
        .drop('y_rec')
    )
    for submission in submissions
]

[0.3926945 0.6073055]


In [73]:
joined_submission_df = submissions_df[0]
for df in submissions_df[1:]:
    joined_submission_df = joined_submission_df.join(df, 'user_id')
joined_submission_df

user_id,w2v_rec,als_rec
str,list[str],list[str]
"""1d664c61-55cb-…","[""cb346b46-0f98-488e-951b-4c9b9c3a32b1"", ""9232a936-ce2d-473f-8526-6da103790f1f"", … ""9e02860b-3bc2-4fda-9afa-ed3a513b12e8""]","[""db2802ef-5aac-4a78-a688-f022944f186b"", ""62c59774-4d7d-4ad5-b46b-e5de9e0730da"", … ""af48c579-07bf-4bab-9755-31d8aaba3511""]"
"""f680183f-5c16-…","[""b306c596-f83f-4007-a4c9-d72ef951e8e8"", ""3bcf5b33-5912-4666-be5e-cbae8d1d363a"", … ""73807b70-4468-4d11-b0b7-9cb094d02826""]","[""b306c596-f83f-4007-a4c9-d72ef951e8e8"", ""92f2e07f-24b5-41bd-9d87-15c355cbba08"", … ""2e5f8198-d76c-40de-8c1a-4fda388086c3""]"
"""ad3dbca0-9eb0-…","[""93015208-5848-4a4b-a090-2a91224e9ccd"", ""a2be2a41-deb9-40ac-83ca-85ea2748aa08"", … ""c521b9a5-c336-4c29-9051-0fa8b88d4043""]","[""5bc90a5d-1356-4958-821c-64328ac567ea"", ""7a6eba6b-e565-4e9c-a383-00eec0c73f5a"", … ""d5d62e25-f4a8-4b4f-bc98-3fe25346d38d""]"
"""218523a8-2e9d-…","[""4b23366c-7fe0-4529-8034-90c8866c3e4e"", ""24962209-ca4e-4a1e-9e85-fd7d1e86a07f"", … ""a2de77b9-83d6-4322-9b33-2d08006ccaf4""]","[""24962209-ca4e-4a1e-9e85-fd7d1e86a07f"", ""78ca3640-59ca-4e18-b218-f8605c2c7344"", … ""1cf28489-fb9a-4732-91c1-609b6faeca1d""]"
"""84f5c9be-99a5-…","[""bd3e86c2-8403-4b26-b057-0c6de8750a2a"", ""1a424e93-7899-416f-9bfc-f21923cc14cc"", … ""0f5319c2-c270-4aa9-8147-7648b0439192""]","[""68acfdf7-29cc-4fb0-bfaf-a1a7eecb8502"", ""f4ff906c-7acd-4235-b62d-d8ffc5fbebc7"", … ""1ca54366-2198-4079-a6c5-f030712fe9f3""]"
"""53d0e85b-588a-…","[""8c2895aa-3204-482a-a5dd-f99bf9880732"", ""e4feec41-ac39-4358-bee5-7608abb8946e"", … ""89b233c3-7eb8-4865-a066-e31839107f5a""]","[""e4feec41-ac39-4358-bee5-7608abb8946e"", ""5fa9bef8-ed42-421a-ab82-5e3ad35c0cbb"", … ""e0e8fa50-748b-4e83-baaf-d5daf051e76d""]"
"""b9a43f13-213a-…","[""40d5f4f7-3560-4638-a562-055013eee0d4"", ""d1947eb3-8fba-4084-a173-a514820ee4cc"", … ""0f971801-697f-4652-8cc3-4c493b8472cd""]","[""c8822a9f-f6f2-4025-ae84-8b7cd45c916f"", ""42cee962-0f50-4728-b887-01cb7a207075"", … ""98e69a29-ee83-41f1-924e-08a50a32efdc""]"
"""e62c7b73-54f1-…","[""98e69a29-ee83-41f1-924e-08a50a32efdc"", ""3e6852cc-d954-443b-a636-74a3a09afe95"", … ""1657c161-1e2d-4823-b9bc-39db72c4b4d6""]","[""02e9fac4-b25a-4779-b51b-cefd2189bd59"", ""1657c161-1e2d-4823-b9bc-39db72c4b4d6"", … ""8caad1cc-e3a0-48f5-af61-89b6025d38e0""]"
"""d7f1921d-a86b-…","[""1206882b-3557-4072-908d-55636329d943"", ""a2be2a41-deb9-40ac-83ca-85ea2748aa08"", … ""3a341c44-0edc-408b-a234-8eaad397cd54""]","[""f3dce478-ec1a-45a8-923d-59ecdbc6c59d"", ""58389edf-72df-468f-8e79-7fb8ce6a6124"", … ""2099bb7f-bcb2-40b6-a58c-77f57d375b7d""]"


In [76]:
submission = []
for user_id, *submission_recs in tqdm(joined_submission_df.rows()):
    item_id_weight = defaultdict(float)

    for i, y_rec in enumerate(submission_recs):
        for pos, item_id in enumerate(y_rec):
            # в качестве веса используем вес позиции в метрике ndcg и метрику качества метода
            item_id_weight[item_id] += submission_weights[i] * 1 / np.log2(pos + 2)

    y_rec = [
        item_id
        for item_id, _ in sorted(item_id_weight.items(), key=lambda x: -x[1])
    ][:TOP_K]
    
    submission.append((user_id, y_rec))
    
submission = pl.DataFrame(submission, schema=('user_id', 'y_rec'))
submission.write_parquet('weighted_submission.parquet')
submission

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 50000/50000 [00:02<00:00, 24688.34it/s]


user_id,y_rec
str,list[str]
"""1d664c61-55cb-…","[""db2802ef-5aac-4a78-a688-f022944f186b"", ""9232a936-ce2d-473f-8526-6da103790f1f"", … ""56e4a99d-cefc-4c87-9484-c6f4ae663c55""]"
"""f680183f-5c16-…","[""b306c596-f83f-4007-a4c9-d72ef951e8e8"", ""92f2e07f-24b5-41bd-9d87-15c355cbba08"", … ""2e5f8198-d76c-40de-8c1a-4fda388086c3""]"
"""ad3dbca0-9eb0-…","[""5bc90a5d-1356-4958-821c-64328ac567ea"", ""a2be2a41-deb9-40ac-83ca-85ea2748aa08"", … ""9716186f-5518-4158-98fa-be86a800c8b7""]"
"""218523a8-2e9d-…","[""24962209-ca4e-4a1e-9e85-fd7d1e86a07f"", ""4b23366c-7fe0-4529-8034-90c8866c3e4e"", … ""7ce5e594-a09f-4526-934a-d52cdcaa689a""]"
"""84f5c9be-99a5-…","[""68acfdf7-29cc-4fb0-bfaf-a1a7eecb8502"", ""bd3e86c2-8403-4b26-b057-0c6de8750a2a"", … ""5342589b-b595-452b-a848-46eb26a46ef8""]"
"""53d0e85b-588a-…","[""e4feec41-ac39-4358-bee5-7608abb8946e"", ""8c2895aa-3204-482a-a5dd-f99bf9880732"", … ""56840eb3-c642-4189-a870-f9adae904b69""]"
"""b9a43f13-213a-…","[""c8822a9f-f6f2-4025-ae84-8b7cd45c916f"", ""40d5f4f7-3560-4638-a562-055013eee0d4"", … ""50bab9e2-d0ab-4952-aaf2-4bd860c8396e""]"
"""e62c7b73-54f1-…","[""02e9fac4-b25a-4779-b51b-cefd2189bd59"", ""3e6852cc-d954-443b-a636-74a3a09afe95"", … ""0c05aab0-5d09-4b78-ae8a-e1272628f201""]"
"""d7f1921d-a86b-…","[""f3dce478-ec1a-45a8-923d-59ecdbc6c59d"", ""58389edf-72df-468f-8e79-7fb8ce6a6124"", … ""577656f3-09f6-4b85-99b8-703f215c9633""]"


Итого, получили метрики NDCG@20 = 0.0835 и Hitrate@20 = 0.367

В блоке про ранжирование мы поговорим, как можно более эффективно объединять рекомендации из нескольких источников с помощью модели ранжирования